In [ ]:
import os
import sys
import random
import numpy as np
import pandas as pd
import scipy.io as scio
import matplotlib.pyplot as plt

from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init
from torch.utils.data import Dataset, DataLoader

In [ ]:
def seed_all(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

seed = 42
seed_all(seed)

In [ ]:
feature_path = "AWA2/split/res101.mat"
data = scio.loadmat(feature_path)
features = data['features'].transpose(1,0)
labels = data['labels'].reshape(-1).tolist()
first_idx = {}
for label_value in range(1,51):
    first_idx[label_value] = labels.index(label_value)
sorted_first_idx = sorted(first_idx.items(), key= lambda x: x[1])
label_orders = [k for k,v in sorted_first_idx]
start_indices = [v for k,v in sorted_first_idx]
end_indices = start_indices[1:] + [len(labels)]
train_indices, test_indices = [], []
train_labels, test_labels = [], []
for label, start_idx, end_idx in zip(label_orders, start_indices, end_indices):
    category_num = end_idx - start_idx
    train_num = int(category_num * 0.6)
    category_train_indices = sorted(random.sample(range(start_idx, end_idx), train_num))
    train_indices.extend(category_train_indices)
    train_labels.extend([label]*len(category_train_indices))
    category_test_indices = list(set(range(start_idx, end_idx)) - set(category_train_indices))
    test_indices.extend(category_test_indices)
    test_labels.extend([label]*len(category_test_indices))
train_features = features[train_indices]
test_features = features[test_indices]

train_features = torch.tensor(train_features, dtype=torch.float32)
test_features = torch.tensor(test_features, dtype=torch.float32)
train_labels = torch.tensor(train_labels).reshape(-1,1)
test_labels = torch.tensor(test_labels).reshape(-1,1)
print(f"train sample num: {len(train_labels)}")
print(f"test sample num: {len(test_labels)}")

In [ ]:
class AWA2(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels
    
    def __getitem__(self, index):
        feature = self.features[index]
        label = self.labels[index]
        return feature, label
    
    def __len__(self):
        return len(self.labels)

In [ ]:
class LinearRegression(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(LinearRegression, self).__init__()
        self.num_layers = len(hidden_dim) + 1 if hidden_dim is not None else 1
        hidden_dim = [] if hidden_dim is None else hidden_dim
        self.layers = nn.ModuleList(nn.Linear(in_channels, out_channels) for in_channels, out_channels in zip([input_dim] + hidden_dim, hidden_dim + [1]))

        self.__init_parameters__()

    def __init_parameters__(self):
    # initialize the parameters of the modules
        for p in self.layers.parameters():
            if p.dim() > 1:
                init.xavier_uniform_(p)

    def forward(self, x):
        for i in range(self.num_layers):
            if i < self.num_layers - 1:
                x = F.relu(self.layers[i](x))
            else:
                x = self.layers[i](x)
        return x

In [ ]:
class LogisticRegression(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes):
        super(LogisticRegression, self).__init__()
        self.num_layers = len(hidden_dim) + 1 if hidden_dim is not None else 1
        hidden_dim = [] if hidden_dim is None else hidden_dim
        self.layers = nn.ModuleList(nn.Linear(in_channels, out_channels) for in_channels, out_channels in zip([input_dim] + hidden_dim, hidden_dim + [num_classes]))

        self.__init_parameters__()

    def __init_parameters__(self):
    # initialize the parameters of the modules
        for p in self.layers.parameters():
            if p.dim() > 1:
                init.xavier_uniform_(p)

    def forward(self, x):
        for i in range(self.num_layers):
            if i < self.num_layers - 1:
                x = F.relu(self.layers[i](x))
            else:
                x = self.layers[i](x)
        return x

In [ ]:
input_dim = 2048
hidden_dim = [1024,512,256,128,64]
num_classes = 50
epochs = 100
lr = 1e-2
weight_decay = 5e-4
batch_size = 32
gpu_order = 0
device = torch.device(f"cuda:{gpu_order}" if torch.cuda.is_available() else "cpu")

train_dataset = AWA2(train_features, train_labels)
test_dataset = AWA2(test_features, test_labels)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
# model = LinearRegression(input_dim=input_dim, hidden_dim=hidden_dim)
# criterion = nn.L1Loss()

# model.to(device)

# optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

# # train model
# best_acc = 0
# for epoch in tqdm(range(epochs)):
#     for batch_features, batch_labels in train_loader:
#         batch_features = batch_features.to(device)
#         batch_labels = batch_labels.to(device)
#         optimizer.zero_grad()
#         output = model(batch_features)
#         loss = criterion(batch_labels, output)
#         loss.backward()
#         optimizer.step()

#     # test model
#     current_acc = 0
#     for batch_features, batch_labels in test_loader:
#         batch_features = batch_features.to(device)
#         batch_labels = batch_labels.to(device)
#         output = (torch.clamp_(model(batch_features),min=0.5, max=50.5) + 0.5).int()
#         current_acc += (output == batch_labels).sum().item()

#     if best_acc < current_acc:
#         best_acc = current_acc
#         torch.save(model, f"linear_regression_{hidden_dim}.pth")

# print(best_acc)

In [ ]:
# model = torch.load(f"linear_regression_{hidden_dim}.pth")
# model.to(device)
# test_loader = DataLoader(dataset=test_dataset, batch_size=1, shuffle=True)

# acc_dict, test_dict = {}, {}
# for i in range(1, 51):
#     acc_dict[i] = test_dict[i] = 0
# for feature, label in test_loader:
#     feature = feature.to(device)
#     label = label.to(device)
#     output = (torch.clamp_(model(feature),min=0.5, max=50.5) + 0.5).int()
#     test_dict[label.item()] += 1
#     if label == output:
#         acc_dict[label.item()] += 1
# print("acc dict:", acc_dict)
# print("test dict:", test_dict)

# rate = []
# acc_count = 0
# for i in range(1, 51):
#     rate.append(acc_dict[i]/test_dict[i])
#     acc_count += acc_dict[i]
# print("acc count:", acc_count)
# canvas = plt.figure()
# plt.bar(range(1, 51), rate)
# plt.ylim(0,1)
# plt.yticks(np.arange(0, 1.2, 0.2))
# plt.xlim(0,51)
# plt.xticks(np.arange(0, 51, 10))
# plt.show()

In [ ]:
model = LogisticRegression(input_dim=input_dim, hidden_dim=hidden_dim, num_classes=num_classes)
criterion = nn.CrossEntropyLoss()

model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

# train model
best_acc = 0
for epoch in tqdm(range(epochs)):
    for batch_features, batch_labels in train_loader:
        batch_features = batch_features.to(device)
        batch_labels = F.one_hot(batch_labels.reshape(-1)-1, 50).float()
        batch_labels = batch_labels.to(device)
        optimizer.zero_grad()
        output = model(batch_features)
        loss = criterion(batch_labels, output)
        loss.backward()
        optimizer.step()

    # test model
    current_acc = 0
    for batch_features, batch_labels in test_loader:
        batch_features = batch_features.to(device)
        batch_labels = batch_labels.to(device)
        output = torch.argmax(F.softmax(model(batch_features), dim=-1), dim=-1).unsqueeze(-1) + 1 
        current_acc += (output == batch_labels).sum().item()
    print(current_acc)
    if best_acc < current_acc:
        best_acc = current_acc
        torch.save(model, f"logistic_regression_{hidden_dim}.pth")

print(best_acc)

In [ ]:
model = torch.load(f"logistic_regression_{hidden_dim}.pth")
model.to(device)
test_loader = DataLoader(dataset=test_dataset, batch_size=1, shuffle=True)

acc_dict, test_dict = {}, {}
for i in range(1, 51):
    acc_dict[i] = test_dict[i] = 0
for feature, label in test_loader:
    feature = feature.to(device)
    label = label.to(device)
    output = torch.argmax(F.softmax(model(feature), dim=-1), dim=-1) + 1
    test_dict[label.item()] += 1
    if label == output:
        acc_dict[label.item()] += 1
print("acc dict:", acc_dict)
print("test dict:", test_dict)

rate = []
acc_count = 0
for i in range(1, 51):
    rate.append(acc_dict[i]/test_dict[i])
    acc_count += acc_dict[i]
print("acc count:", acc_count)
canvas = plt.figure()
plt.bar(range(1, 51), rate)
plt.ylim(0,1)
plt.yticks(np.arange(0, 1.2, 0.2))
plt.xlim(0,51)
plt.xticks(np.arange(0, 51, 10))
plt.show()